# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2019", "end" : "2023"
          }

env = Env(**config)

agentIds = [1, 2, 3, 4]
env.config_agents(agentIds)

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}
params_2 = {"agentId" : 2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}
params_3 = {"agentId" : 3, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"MOMENTUM"}
params_4 = {"agentId" : 4, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

In [ ]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

agent_3 = Agent(**params_3)
agent_3.update_policy_params(params = 5)
agent_3.run_episode()

agent_4 = Agent(**params_4)
agent_4.update_policy_params(params = (7, 14, 21))
agent_4.run_episode()

In [ ]:
#env.update_indicator()

In [ ]:
env.get_viz(agentId=1, symbol = "BTC")

## Global Report

In [ ]:
env.globalReport()

In [ ]:
0

#  MultiAgent System

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }
env = Env(**config)


def custom_action():
    print(" *** Barrier passee *** ")


agentId_1 = 1
agentId_2 = 2
agentId_3 = 3
agentId_4 = 4
agentID = [1, 2, 3, 4]
env.config_agents(agentID)

agent_bus = {
    "running" : {agentId_1 : False, agentId_2 : False, agentId_3 : False, agentId_4 : False},
    "msg" : {agentId_1 : None, agentId_2 : None, agentId_3 : None, agentId_4 : None},
    "stop" : False
}
master_bus = {
    "agent" : {agentId_1 : None, agentId_2 : None, agentId_3 : None, agentId_4 : None},
    "activeAgent" : None,
    "stop" : False
}

activeAgent = agentId_1

N = len(agentID)
condition = Condition()
barrier = Barrier(N, action=custom_action)

boss_params = {"condition" : condition, "activeAgent" : activeAgent ,  "agent_bus" : agent_bus, "master_bus" : master_bus}

params_1 = {"agentId" : agentId_1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_2 = {"agentId" : agentId_2, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_3 = {"agentId" : agentId_3, "symbol" : "SOL", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}

params_4 = {"agentId" : agentId_4, "symbol" : "EGLD", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_bus" : agent_bus, "master_bus" : master_bus}


Boss = MasterAgentThread(**boss_params)
Boss.addAgent(agentId_1)
Boss.addAgent(agentId_2)
Boss.addAgent(agentId_3)
Boss.addAgent(agentId_4)

agent_1 = MAgentThread(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

agent_3 = MAgentThread(**params_3)
agent_3.update_policy_params(params = 6)

agent_4 = MAgentThread(**params_4)
agent_4.update_policy_params(params = (14, 21, 36))


In [ ]:
Boss.start()
agent_1.start()
agent_2.start()
agent_3.start()
agent_4.start()

agent_1.join()
agent_2.join()
agent_3.join()
agent_4.join()
Boss.join()

In [ ]:
env.get_viz(agentId=4, symbol = "EGLD")

In [ ]:
env.globalReport()

In [ ]:
d = {x : None for x in range(4)}
d

In [ ]:
from plotly.figure_factory import create_distplot
import numpy as np
import pandas as pd

df = pd.DataFrame({'2012': np.random.randn(200), 
                   '2013': np.random.randn(200)+1}
                  )
fig = create_distplot([df[c] for c in df.columns], df.columns)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = np.random.randn(1000)
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = np.random.randn(200) - 2
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 2
x4 = np.random.randn(200) + 4

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200)
x2 = np.random.randn(200) + 2

group_labels = ['Group 1', 'Group 2']

colors = ['slategray', 'magenta']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.5,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200) - 1
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 1

hist_data = [x1, x2, x3]

group_labels = ['Group 1', 'Group 2', 'Group 3']
colors = ['#333F44', '#37AA9C', '#94F3E4']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text='Curve and Rug Plot')
fig.show()

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
my_list = [1, 2, 3, 4, 5]
iterable = iter(my_list)

for x in iterable:
    print(x)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()